In [81]:
#!pip install geopy
#!pip install folium
#!pip install openrouteservice
#!pip install gmaps
#!pip install convert
import convert
import openrouteservice
import pandas as pd
from geopy.distance import geodesic
import folium


def charging_stat_plot(source, destination):
    # Read CSV file with EV charger locations
    df = pd.read_csv('ev_stations_v1.csv')
    
    # Filter chargers for Seattle
    chargers_df = df[df['State'] == 'WA']
    
    # Define origin and destination points (longitude, latitude)
    #origin = (47.6205, -122.3493)  # Example coordinates for Seattle downtown
    #destination = (47.6272, -122.28783)  # Example coordinates for Seattle downtown
    origin = source
    destination = destination
    
    # Define function to calculate distance between two points in miles
    def calculate_distance(coord1, coord2):
        return geodesic(coord1, coord2).miles
    
    api_key = '5b3ce3597851110001cf6248ddd53f33b0ef411ab16b3ddf0d255df4'
    client = openrouteservice.Client(key=api_key)
    
    
    #opensource requires (long,lat) format 
    start_coords = (origin[1],origin[0])
    end_coords = (destination[1],destination[0])
    
    #Getting the route coordinates
    geometry = client.directions([start_coords,end_coords])['routes'][0]['geometry']
    decoded = openrouteservice.convert.decode_polyline(geometry)

    #Calculating the mid-points
    mid_point = decoded['coordinates'][int(len(decoded['coordinates'])/2)]
    
    # Calculate distances between origin and each charger location
    chargers_df['distance_from_origin'] = chargers_df.apply(lambda row: calculate_distance((row['Latitude'], row['Longitude']), origin), axis=1)
    
    # Calculate distances between destination and each charger location
    chargers_df['distance_from_destination'] = chargers_df.apply(lambda row: calculate_distance((row['Latitude'], row['Longitude']), destination), axis=1)
    
    # Calculate distances between midpoint and each charger location
    chargers_df['distance_from_mid_point'] = chargers_df.apply(lambda row: calculate_distance((row['Latitude'], row['Longitude']), (mid_point[1],mid_point[0])), axis=1)
    
    
    
    # Calculate distance between origin and destination
    distance_between_origin_midpoint = calculate_distance(origin, (mid_point[1],mid_point[0]))
    # Calculate radius as half the distance between origin and destination
    distance_between_origin_destination = calculate_distance(origin, destination)
    
    #Radius 
    radius = distance_between_origin_destination / 2
    
    #Picking all the data 
    chargers_within_radius = chargers_df[(chargers_df['distance_from_origin'] <= radius) | (chargers_df['distance_from_destination'] <= radius) | (chargers_df['distance_from_mid_point'] <= radius)]
    #chargers_within_radius = chargers_df[(chargers_df['distance_from_mid_point'] <= radius)]
    
    ## Create map centered on origin
    m = folium.Map(location=origin, zoom_start=14)
    
    #Route layer
    folium.GeoJson(decoded).add_to(m)
    
    ## Add origin marker to the map
    folium.Marker(location=origin, popup='Origin', icon=folium.Icon(color='blue')).add_to(m)
    #
    ## Add destination marker to the map
    folium.Marker(location=destination, popup='Destination', icon=folium.Icon(color='blue')).add_to(m)
    
    ## Add midpoint marker to the map
    folium.Marker(location=mid_point, popup='Mid', icon=folium.Icon(color='blue')).add_to(m)
    #
    ## Add chargers within radius markers to the map
    for index, charger in chargers_within_radius.iterrows():
        folium.Marker(location=[charger['Latitude'], charger['Longitude']], popup=charger['Station Name'], icon=folium.Icon(color='red')).add_to(m)
    
    # Display the map
    return m

charging_stat_plot([47.6205, -122.3493],[47.6272, -122.28783])


/var/folders/12/wrdf3scj0fb6r95lfqzfcbqr0000gn/T/ipykernel_15762/2638586662.py:16: DtypeWarning: Columns (6,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('ev_stations_v1.csv')
/var/folders/12/wrdf3scj0fb6r95lfqzfcbqr0000gn/T/ipykernel_15762/2638586662.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chargers_df['distance_from_origin'] = chargers_df.apply(lambda row: calculate_distance((row['Latitude'], row['Longitude']), origin), axis=1)
/var/folders/12/wrdf3scj0fb6r95lfqzfcbqr0000gn/T/ipykernel_15762/2638586662.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu